# This section:

# *1. Uses FAISS to retrieve the most semantically relevant transcript chunks based on a user's question,*

# *2. Constructs a context prompt from the retrieved chunks and sends it to OpenAI's GPT model via the chat API,*

# *3. Generates a grounded answer using Retrieval-Augmented Generation (RAG), ensuring the response is based solely on the transcript content rather than general model knowledge.*



### This section implements a Retrieval-Augmented Generation (RAG) pipeline — combining transcript chunk embeddings with OpenAI's GPT model.

### When a user asks a question:

### 1. It finds the most relevant transcript chunks using semantic search (via FAISS).

### 2. It formats those chunks as context and sends them along with the question to GPT.

### 3. GPT is instructed to only answer based on the provided text, ensuring that the answer is grounded in the actual video transcripts.

### This approach enables accurate and context-aware answers tied directly to the processed transcripts.

In [1]:
# --- Part 5: RAG + Topic-Aware Retrieval from Qdrant ---
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue
from sentence_transformers import SentenceTransformer
from openai import OpenAI
import pandas as pd
from datetime import timedelta

In [2]:
# --- Load embedding model and OpenAI client ---
model = SentenceTransformer('all-MiniLM-L6-v2')
client = OpenAI(api_key="")  # Replace with your secure key or env var

# --- Qdrant setup ---
collection_name = "video_chunks"
client_qdrant = QdrantClient(host="localhost", port=6333)

# --- Function: Search with optional topic filter ---
def retrieve_chunks(question, topic_filter=None, top_k=3):
    question_embedding = model.encode([question]).astype('float32')[0]

    qdrant_filter = None
    if topic_filter is not None:
        qdrant_filter = Filter(
            must=[FieldCondition(key="topic", match=MatchValue(value=topic_filter))]
        )

    search_result = client_qdrant.search(
        collection_name=collection_name,
        query_vector=question_embedding,
        limit=top_k,
        query_filter=qdrant_filter
    )

    chunks = []
    for hit in search_result:
        payload = hit.payload
        chunks.append({
            "video_id": payload["video_id"],
            "chunk_id": payload["chunk_id"],
            "text": payload["text"],
            "start": payload["start"]
        })
    return pd.DataFrame(chunks)

In [3]:
# --- Function: RAG answer using OpenAI ---
def generate_answer(question, chunks_df):
    context = "\n\n".join([f"[{row.video_id} - chunk {row.chunk_id}]\n{row.text}" for _, row in chunks_df.iterrows()])
    
    system_prompt = (
        "You are a helpful assistant. Use only the following context to answer the question.\n"
        "If unsure, say 'I don’t know.'\n\n"
        f"Context:\n{context}"
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        temperature=0
    )
    return response.choices[0].message.content

In [4]:
# --- Example usage ---
question = "How does reinforcement learning relate to exploration?"
topic_number = 5  # (Optional) Set this to filter by a specific BERTopic topic

retrieved_df = retrieve_chunks(question, topic_filter=topic_number, top_k=3)
answer = generate_answer(question, retrieved_df)

# --- Print results ---
print("\nAnswer:\n", answer)
print("\nRetrieved Chunks:")
for _, row in retrieved_df.iterrows():
    start_time = int(row["start"])
    video_url = f"https://www.youtube.com/watch?v={row.video_id}&t={start_time}s"
    print(f"\n[Chunk {row.chunk_id} from {row.video_id}] — Start: {timedelta(seconds=start_time)}")
    print(f"Jump to: {video_url}")
    print(row.text)

C:\Users\algba\AppData\Local\Temp\ipykernel_24540\2604927637.py:19: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client_qdrant.search(



Answer:
 Reinforcement learning involves an agent learning to make decisions by interacting with an environment and receiving rewards or penalties based on its actions. Exploration is a crucial aspect of reinforcement learning, as the agent needs to try out different actions to discover the best strategy for maximizing its rewards. By exploring different actions and observing the outcomes, the agent can learn which actions lead to positive rewards and adjust its decision-making accordingly.

Retrieved Chunks:

[Chunk 36 from t9zxmEHGT1s] — Start: 0:13:42
Jump to: https://www.youtube.com/watch?v=t9zxmEHGT1s&t=822s
from left to right uh so the whole agent is uh quite considerable uh uron Network so this is a uh a deep Network that accepts uh the uh row information uh or the sensing information the X under bar that we have seen earlier and uh directly spits out the uh steering Direction and the acceleration deceleration Alpha so now we have an end to endend uh system that uh is doing all